In [6]:
from scipy import misc
import tensorflow as tf
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from PIL import Image
from sklearn import preprocessing



batch_size = 1
class_num = 34

fileNameQue = tf.train.string_input_producer(["tfrecords/train.tfrecords"])
reader = tf.TFRecordReader()
key,value = reader.read(fileNameQue)
features = tf.parse_single_example(value,features={ 'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string)})

image = tf.decode_raw(features["img_raw"], tf.uint8)
#归一化
image=tf.cast(image,tf.float64)*(1/255)
image=tf.reshape(image,[48,24,3])
#图片转变为黑白图片
image=tf.split(image,3,2)[0]

label=features['label']
label = tf.cast(label, tf.int32)
# enc = preprocessing.OneHotEncoder()
# enc.fit([label])
# enc.transform([[0, 1, 3]]).toarray()
 
init = tf.global_variables_initializer()
# 随机选取图片形成一个ｂａｔｃｈ
imageBatch, labelBatch = tf.train.shuffle_batch([image, label], \
                        batch_size = 10,capacity = 2000, min_after_dequeue = 1900)

with tf.Session() as sess:

    sess.run(init)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for i in range(2):
#         img, lab = sess.run([image, label])
#         print(img.shape)
#         print(lab)
        img,lab=sess.run([imageBatch, labelBatch])
        print(lab.shape)
#         print(img.shape)
#         img=img[i].reshape([48,24])
        for j in range(10):
#             print(img.shape)
            im = img[j].reshape([48,24])
            print(lab[j])
#             print(img.shape)
#             print(img[i].shape)
#             print(im.shape)
            result = Image.fromarray(im,'L')

            plt.imshow(im)
            plt.show()


            result.save('./train_images/'+str(i) +'_'+str(lab)+ '.png', 'png')
        print('----------------------------------------------------------------------------------')

    coord.request_stop()
    coord.join(threads)

ValueError: setting an array element with a sequence.